In [1]:
%load_ext rpy2.ipython

In [60]:
%%R
library(phyloseq)
library(DESeq2)
library(dplyr)
library(magrittr)

In [26]:
%%R
sd = read.table("data/ABCD_combined_reads_mapper.tsv",
                comment = "",
                sep = "\t",
                header = TRUE, 
                stringsAsFactors = FALSE)

rownames(sd) = sd$X.SampleID

ps = import_biom("data/otu_table_wtax.biom")
sample_data(ps) = sample_data(sd)
md = sample_data(ps)

density.min = 1.7125
density.max = 1.755

ps.int = prune_samples((md$Density > density.min)&
              (md$Density < density.max)&
              (md$Treatment %in% c("12CCPS","13CCPS","12CCSS","13CCSS"))&
              (md$Day == 30)&
              !(grepl("bulk", md$X.SampleID)), ps)

ps.int

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 6236 taxa and 34 samples ]
sample_data() Sample Data:       [ 34 samples by 7 sample variables ]
tax_table()   Taxonomy Table:    [ 6236 taxa by 8 taxonomic ranks ]


In [44]:
%%R
md.int = sample_data(ps.int)
tmp = matrix(unlist(strsplit(md.int$X.SampleID, "[.]")), ncol=4, byrow = TRUE)
name.df = data.frame(tmp)
colnames(name.df) = c("contVexp", "SSVPS", "Day", "Fraction")
md.int$contVexp = name.df$contVexp
md.int$SSVPS = name.df$SSVPS
sample_data(ps.int) = md.int

In [51]:
%%R
dds = phyloseq_to_deseq2(ps.int, ~ contVexp * SSVPS)
dds = DESeq(dds)

converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
-- replacing outliers and refitting for 4 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing


In [70]:
%%R
int.df = results(dds)
int.df$OTU = rownames(int.df)
int.df = data.frame(int.df)
int.df %<>%
    arrange(desc(abs(log2FoldChange))) %>%
    select(pvalue, log2FoldChange, OTU) %>%
    rename(int.p = pvalue, int.l2fc = log2FoldChange)

write.csv(int.df, "data/interaction.csv", row.names = FALSE)

In [71]:
!head data/interaction.csv

"int.p","int.l2fc","OTU"
0.000196020907404688,-3.17526409521004,"OTU.1414"
0.00148670570368743,-3.1303528970411,"OTU.1051"
0.00162140275175015,3.09851302220106,"OTU.500"
0.00124207172844057,3.05592227084979,"OTU.228"
0.00217187913649674,2.97985443000639,"OTU.298"
0.00243326930471977,2.95714102628657,"OTU.1324"
0.00252361312553193,2.86137768292596,"OTU.630"
0.00330184832267835,-2.81926051446317,"OTU.1594"
0.000412159240455176,-2.7763306855473,"OTU.206"
